# Residual Networks !!!

> Why there is need of residual connection ?

- as the depth of cnn model increases the performace keeps getting better.

- Ex. VGG(18-layers) beats Alexnet(8-layers) performace. 

- but after increaing more layer in VGG its start decreasing its performace.

- residaul connection helps in improving training more larger models.

- without loosing its performace even after increasing its depth.

- but using residual connection can cuase exponential increase of magnitude of activation at initilization. 

- to control and re-centere the magnitude of actiation.

- it uses batch-norm at each layer.(residual block)

> Why model with more depth start decreasing its performance ? 

- so the model with no skip or residual connetion is known as seueantial model.

- Ex. layer = l1, l2, l3 and l4 -> model_with_ln -> y =  (l4(l3(l2(l1(x))))) (forward pass in seq model look like this right ?)

- in a deep neural network even a small change in a input can cuase lost of change in its gradient. (also konwn as shattered gradeint phenomenon)

- due to this gradient descent algo(gda) doesn't perform well cuase gda needs smooth gradient curve to perform well.

- cuase the performaance of gda is depend on before and after gradient.(due to chain rule)

- check out the backprop file to understand this more.

> residual connections

# will complete tmwr(first lets get done with code)





In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import torch
from torch import nn 
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):
    # transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010],
        ),
    ])

    if test:
        dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        return data_loader

    # load the dataset
    dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
    

    num_train = len(dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

    return train_loader, valid_loader

# CIFAR10 dataset
train_loader, valid_loader = data_loader(data_dir='./data', batch_size=64)
test_loader = data_loader(data_dir='./data', batch_size=64, test=True)


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1, downsample: nn.Module = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = self.relu(out)
        
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer0 = self._build_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._build_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._build_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._build_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _build_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [ ]:
num_classes = 10
num_epochs = 50
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.001)

# Train the model
total_step = len(train_loader)

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
import torch
import wandb
import gc
from torch.cuda.amp import autocast, GradScaler
import wandb

# Initialize wandb
wandb.init(project="resnet")

scaler = GradScaler()
total_train_steps = len(train_loader)
total_val_steps = len(test_loader)

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images, labels = images.to(device), labels.to(device)

        # Forward pass and backward pass with AMP
        optimizer.zero_grad()
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Accumulate loss
        running_loss += loss.item()

        

    # Compute the average loss over the epoch
    epoch_loss = running_loss / total_train_steps
    wandb.log({"train_loss": epoch_loss}, step=epoch)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')

    # Validation step
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast():
                outputs = model(images)
                val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item()

    # Compute the average validation loss over the epoch
    epoch_val_loss = val_running_loss / total_val_steps
    wandb.log({"val_loss": epoch_val_loss}, step=epoch)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

# Finish wandb run
wandb.finish()


# lets make inference fast with TensorRT